<a href="https://colab.research.google.com/github/acl-jianghao/Langchain-Scripts/blob/main/langchain_vectordb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
%pip install openai --upgrade
%pip install langchain --upgrade
%pip install tiktoken --upgrade
%pip install colab-env --upgrade
%pip install pypdf --upgrade

%pip install faiss-cpu
%pip install chromadb
%pip install pinecone-client

In [103]:
import os
import colab_env
colab_env.__version__

'0.2.0'

In [104]:
import openai
from langchain.document_loaders import PyPDFLoader

In [39]:
# !wget -P /content/gdrive/MyDrive/data/ -U NoSuchBrowser/1.0 https://arxiv.org/pdf/2307.00386.pdf

In [40]:
!ls -alt /content/gdrive/MyDrive/data/

total 475
-rw------- 1 root root 486144 Jul  4 01:25 2307.00386.pdf


In [46]:
# refer to:

loader = PyPDFLoader('/content/gdrive/MyDrive/data/2307.00386.pdf')
docs = loader.load()

In [90]:
print("There are %s document(s)."% len(docs))
print("There are %s characters in first document(page)." % len(docs[0].page_content))

There are 8 document(s).
There are 4897 characters in the first document(page).


In [91]:
# refer to:

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
split_docs = text_splitter.split_documents(docs)

In [92]:
print("Now the PDF file splitted in %s documents." % len(split_docs))
print("There are %s characters in first splitted document." % len(split_docs[0].page_content))

Now the PDF file splitted in 40 documents.
There are 999 characters in first splitted document.


In [58]:
# openai.api_key = os.getenv('OPENAI_API_KEY')

In [83]:
from langchain.vectorstores import FAISS, Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

import pinecone

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [61]:
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))

Local vector index, sample codes implementted by Faiss, Chroma an Pincone vector db.

In [106]:
os.getenv('PINECONE_API_KEY')

In [101]:
index_root_dir = '/content/gdrive/MyDrive/index/'

# db_faiss = FAISS.from_documents(split_docs, embeddings)
# db_faiss.save_local(index_root_dir+'faiss_index')


# db_chroma = Chroma.from_documents(
#     split_docs, embeddings, persist_directory=index_root_dir+'chroma_index')
# db_chroma.persist()

pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),  # find at app.pinecone.io
    environment=os.getenv('PINECONE_ENV'),  # next to api key in console
)

db_pinecone = Pinecone.from_documents(docs, embeddings, index_name='test-index')


TypeError: ignored

In [72]:
index_faiss = FAISS.load_local(index_root_dir+'faiss_index', embeddings)
index_chroma = Chroma(persist_directory=index_root_dir+'chroma_index', embedding_function=embeddings)

In [76]:
question = "What is the density dependent model?"
answer_docs_faiss = index_faiss.similarity_search(question);
answer_docs_chroma = index_chroma.similarity_search(question);

In [78]:
print(len(answer_docs_faiss), answer_docs_faiss)
print(len(answer_docs_chroma), answer_docs_chroma)

4 [Document(page_content='m∗\nb,d=mb,d−gσb,dσ0, (11)\nand\nµ∗\nb,d=µb,d−gωb,dω0−gρb,dI3b,dρ03−gϕb,dϕ0−Σr,(12)\n(13)\nrespectively, with I3b,d, the isospin projection. The baryon\ndensity is given by\nnb,d=γb,dZd3k\n(2π)3, (14)\nand the scalar density is\nns\nb,d=γb,dZd3k\n(2π)3m∗\nb,d\nEb,d, (15)\nwhere γb,dis the particle degeneracy (2 for spin 1/2 particles\nand 4 for spin 3/2 particles) and Eb,d=q\nk2+m∗2\nb,dis the\nsingle particle energy.\n2.2 Density Dependent Quark Model\nIn this work, we assume that the deconfined matter phase in\nthe inner core of the compact star is composed of electrons (e)\nand quarks (up ( u), down ( d), and strange ( s)). In this phase,the matter will be described by the DDQM model expressed\nas\nmi=mi0+mI, (16)\nwhere mi0(i=u, d, s ) is the current quark mass and mIis\nthe density dependent quantity which includes the interac-\ntions of the quarks. The motivation for the DDQM approach\nis to include the interactions between quarks in a simple way.\nIt is